<a href="https://colab.research.google.com/github/sagunkayastha/CAI_Workshop/blob/main/Workshop_4/Deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Change to gpu runtime

# Standard Libraries
import os
import numpy as np
import pandas as pd
import random as rn

# Visualization libraries

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style({"axes.facecolor": ".95"})

# Modeling and Machine Learning
from IPython.display import Image
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
# from sklearn.externals.six import StringIO
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz


# Seed for reproducability
seed = 1234
np.random.seed(seed)
rn.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
!wget https://raw.githubusercontent.com/sagunkayastha/CAI_Workshop/main/Workshop_3/Inputs/test.csv
!wget https://raw.githubusercontent.com/sagunkayastha/CAI_Workshop/main/Workshop_3/Inputs/train.csv

In [ ]:
# Specify Paths for easy dataloading

TRAIN_PATH = 'train.csv'
TEST_PATH = 'test.csv'

# Load in training and testing data
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
concat_df = pd.concat([train_df, test_df])

In [ ]:
# Visualize target distribution
train_df['label'].value_counts().sort_index().plot(kind='bar', figsize=(10, 6), rot=0)
plt.title('Visualization of class distribution for the MNIST Dataset', fontsize=20, weight='bold')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Class', fontsize=16)
plt.ylabel('Frequency', fontsize=16);

## Creating single neuron

In [ ]:

def sigmoid(x):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-x))

def single_neuron_forward(inputs, weights, bias):
    """Forward pass through a single neuron using a loop for the weighted sum."""
    total = 0
    for input, weight in zip(inputs, weights):
        total += input * weight
    # total = np.dot(inputs, weights) + bias


    total += bias
    return sigmoid(total)

!['inp'](https://raw.githubusercontent.com/sagunkayastha/CAI_Workshop/a4714e1d1c59d5fff5289ed950097251e119f6cc/Workshop_4/images/single_neuron_example.png)

In [ ]:
# Number of inputs to the neuron
num_inputs = 3

# Initialize weights and bias to random values
weights = [0.4,0.9,-0.11]
bias = [0.1]

# Example input
inputs = np.array([0.5, -0.2, 0.1])
y_true = 1.12

y_pred = single_neuron_forward(inputs, weights, bias)
print("Output:", y_pred)

Binary Cross-Entropy Loss=−(y⋅log(p)+(1−y)⋅log(1−p))

In [ ]:
def loss_function(y_true, y_pred):
    """Binary Cross-Entropy loss function."""
    return -np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))


In [ ]:
def sigmoid_derivative(x):
    """Derivative of the sigmoid function."""
    return sigmoid(x) * (1 - sigmoid(x))

def binary_cross_entropy_derivative(y_true, y_pred):
    """Derivative of binary cross-entropy loss."""
    return -(y_true / y_pred) + ((1 - y_true) / (1 - y_pred))

def update_weights(inputs, weights, bias, y_true, y_pred, learning_rate):
    """Update weights and bias using gradient descent."""
    error = binary_cross_entropy_derivative(y_true, y_pred)
    sigmoid_deriv = sigmoid_derivative(y_pred)

    # Gradient with respect to each weight
    weights_gradient = inputs * error * sigmoid_deriv

    # Gradient with respect to bias
    bias_gradient = error * sigmoid_deriv

    # Update weights and bias
    weights = weights - learning_rate * weights_gradient
    bias = bias -  learning_rate * bias_gradient
    return weights, bias


# Update weights
print(f'loss before: {loss_function(y_true, y_pred)}')
print(f"Initial weights: {weights}")
weights, bias = update_weights(inputs, weights, bias, y_true, y_pred, learning_rate=0.001)
print(f'Updated weights: {weights}')

y_pred = single_neuron_forward(inputs, weights, bias)
print(f'loss after: {loss_function(y_true, y_pred)}')



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical, plot_model

In [ ]:
# Get all pixel features
features = [col for col in train_df.columns if col.startswith('pixel')]
# Split up training to for validation
X_train, X_val, y_train, y_val = train_test_split(train_df[features],
                                                  train_df['label'],
                                                  test_size=0.25,
                                                  random_state=seed)
y_train = y_train.values

In [ ]:
y_train[10]

In [ ]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [ ]:

plt.imshow(X_train.iloc[21].values.reshape(28,28), cmap='gray')
y_train[21]

In [ ]:


# Define the model
model = Sequential([

    Dense(784, input_shape=(784,), activation='relu'),
    # Second hidden layer with a specified number of neurons (e.g., 100)
    Dense(100, activation='relu'),
    # Output layer for multi-class classification
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


In [ ]:
# convert to one-hot vector

# y_test = to_categorical(y_test)

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=8, shuffle=True, validation_data = (X_val, y_val) )

In [ ]:
plot_model(model, to_file='mlp-mnist.png', show_shapes=True)